In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/ING3/NLP/Projet_NLP_chatbot/Projet_VerFinal/dataset'

/content/drive/.shortcut-targets-by-id/1c-DUi14_W785-FFX1R46bJ01W69NheeB/Projet_NLP_chatbot/Projet_VerFinal/dataset


# Import

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ce2e765727ff662df5b222b5f23dec4c90b58760a3b52c0d8c087bbe17a4ebda
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
modelBert = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
import pandas as pd
dataset = pd.read_csv('wiki_movie_plots_embedding.csv')

In [ ]:
print(dataset.keys())

dataset.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

Index(['Unnamed: 0', 'Release Year', 'Title', 'Origin/Ethnicity', 'Director',
       'Cast', 'Genre', 'Wiki Page', 'Plot', 'Plot_Embedding'],
      dtype='object')


In [ ]:
dataset.head()

,Unnamed: 0,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Plot_Embedding
0,0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",[ 1.35721848e-03 2.60153543e-02 -1.53429396e-...
1,1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",[ 1.15494728e-02 7.49101862e-02 7.63061643e-...
2,2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",[-1.93726048e-02 4.27587181e-02 -4.92642820e-...
3,3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,[ 1.81851219e-02 2.28143688e-02 4.11286801e-...
4,4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,[-3.62669602e-02 -1.18787279e-02 4.69595380e-...


# MODEL : K films avec la fonction de calcul de similarité BERT


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_k_films_bert_similarity_with_user_desc(model, dataset, plot, k=5):
    # Encodage du plot du film référence
    reference_embedding = model.encode([plot])

    # Maintenant on doit essayer de trouver les k films les plus proches selon les embeddings
    # Liste pour stocker les informations des k films les plus similaires
    top_k_movies_info = []

    for line in dataset.index:
        # Récupération des embeddings des plots
        plot_embedding = dataset.loc[line, 'Plot_Embedding']
        plot_embedding = np.fromstring(plot_embedding[1:-1], sep=' ')

        # Calcul de la similarité cosine entre la référence et le plot du film
        similarity_score = cosine_similarity(reference_embedding, plot_embedding.reshape(1, -1))[0][0]

        # Stockage des informations des films et du score de similarité
        movie_info = {
            'Title': dataset.loc[line, 'Title'],
            'Genre': dataset.loc[line, 'Genre'],
            'Director': dataset.loc[line, 'Director'],
            'Release Year': dataset.loc[line, 'Release Year'],
            'Similarity Score': similarity_score
        }

        # Ajout des films dans la liste
        top_k_movies_info.append(movie_info)

    # Tri de la liste par scores de similarité décroissants et récupération des k films avec le score le plus élevé
    top_k_movies_info = sorted(top_k_movies_info, key=lambda x: x['Similarity Score'], reverse=True)[:k]

    # Affichage des films dans la console
    # Pour le debugage
    # print(top_k_movies_info)

    # Renvoi des k films à recommander
    return top_k_movies_info

# Fonctions de filtrage

## Filtre de la demande (description ou titre)

In [ ]:
import re

def description_or_title(user_prompt):
    #savoir si l'utilisateur va rentrer une description ou un titre de réference
    #avec regex

    if re.search(r'\btitle\b',user_prompt) or re.search(r'\breference\b',user_prompt):
      return "title"
    if re.search(r'\bdescription\b',user_prompt):
      return "description"


In [ ]:
text = str(input("Do you prefer to find a movie based on a description or a reference title?: "))
filter = description_or_title(text)
print(filter)

Do you prefer to find a movie based on a description or a reference title?: a title
title


## Filtre par rapport au titre

In [ ]:
def filter_by_reference_title(dataset, user_title_ref):
    matching_titles = dataset[dataset['Title'].str.contains(user_title_ref, case=False)]
    matching_ids = []

    if not matching_titles.empty:
        matching_ids = matching_titles['ID'].tolist()
        print(f"Film exist in dataset. IDs: {matching_ids}")
        # Tu peux accéder aux informations sur les films correspondants dans matching_titles
    else:
        print("Film doesn't exist in the dataset")

    return matching_ids[0]

## Filtre par rapport au genre

In [ ]:
def filter_by_genre_film(dataset, user_genre_ref_list):
    # Lister les genres qui sont disponibles dans le dataset
    genres_available = dataset['Genre'].unique().tolist()

    genre_list = user_genre_ref_list.split(',')
    for genre in genre_list:
      if genre in genres_available:
          filtered_dataset = dataset[dataset['Genre'].str.contains(genre, case=False)]
          return filtered_dataset
      else:
        return dataset

## Filtre par rapport à l'année de sortie du film

In [ ]:
unique_year = dataset["Release Year"].unique()
print(unique_year[:25])

[1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914
 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925]


In [ ]:
import pandas as pd
import spacy
import re

nlp = spacy.load('en_core_web_sm')
dataset = pd.read_csv('wiki_movie_plots_deduped.csv')


# FILTRE PAR DATE
def filter_by_year(dataset, operator, year):
    subset = dataset
    if operator == '=':
        subset = dataset[dataset['Release Year'] == year]
    elif operator == '>':
        subset = dataset[dataset['Release Year'] > year]
    elif operator == '<':
        subset = dataset[dataset['Release Year'] < year]
    return subset


def filter_between_two_dates(dataset, list_of_dates):
    # On trie la liste des dates qui contient deux dates
    list_of_dates = sorted(list_of_dates)
    print(list_of_dates)
    subset = dataset[(dataset['Release Year'] >= list_of_dates[0]) & (dataset['Release Year'] <= list_of_dates[1])]
    return subset


def get_information_operator_and_year_from_prompt(dataset, user_prompt):
    filtered_data = dataset
    # Variables qui vont nous permettre de filtrer notre dataset
    year_list = []
    doc = nlp(user_prompt)

    for ent in doc.ents:
        if ent.label_ == 'DATE':
            # Si la phrase contient "between", extrait les années individuelles
            if 'between' in ent.text:
                # Utilise une expression régulière pour extraire les années
                years = re.findall(r'\b\d+\b', ent.text)
                year_list.extend(map(int, years))
            else:
                # Sinon, ajoute simplement l'année à la liste
                year_list.append(int(ent.text))

    # pour debugage
    print("Identified dates:")
    print(year_list)

    # Utiliser les regex pour déterminer l'opérateur à utiliser
    if re.search(r'\b(?:released|directed) in\b', user_prompt) and len(year_list) == 1:
        operator = '='
        filtered_data = filter_by_year(dataset, operator, year_list[0])

    elif re.search(r'\bafter\b', user_prompt) and len(year_list) == 1:
        operator = '>'
        filtered_data = filter_by_year(dataset, operator, year_list[0])

    elif re.search(r'\bbefore\b', user_prompt) and len(year_list) == 1:
        operator = '<'
        filtered_data = filter_by_year(dataset, operator, year_list[0])

    elif re.search(r'\bbetween\b', user_prompt) and len(year_list) == 2:
        filtered_data = filter_between_two_dates(dataset, year_list)

    return filtered_data

In [ ]:
text = str(input("In which released year are you interested?: "))

# Exemple d'utilisation
subset = get_information_operator_and_year_from_prompt(dataset, text)
print(subset.head())


In which released year are you interested?: directed in 2001
Identified dates:
[2001]
       Release Year                          Title Origin/Ethnicity  \
13782          2001                     15 Minutes         American   
13783          2001        3000 Miles to Graceland         American   
13784          2001   A.I. Artificial Intelligence         American   
13785          2001     The Affair of the Necklace         American   
13786          2001                            Ali         American   

                  Director                                               Cast  \
13782        John Herzfeld       Robert De Niro, Edward Burns, Kelsey Grammer   
13783  Demian Lichtenstein  Kurt Russell, Kevin Costner, Courteney Cox, Ch...   
13784     Steven Spielberg  Jude Law, Haley Joel Osment, William Hurt, Fra...   
13785        Charles Shyer         Hilary Swank, Jonathan Pryce, Adrien Brody   
13786         Michael Mann  Will Smith, Jamie Foxx, Jon Voight, Mario Van ...   



In [ ]:
unique_year2 = subset["Release Year"].unique()
print(unique_year2[:25])

[2001]


# Extraction des informations de la query de l'utilisateur

In [ ]:
import spacy
import re
nlp = spacy.load('en_core_web_sm')

# Fonction d'extraction des informations données par l'utilisateurs
# Si l'utilisateur skip le filtrage par genre et année
# On essaye de récupérer le genre et l'année à partir d'expressions régulières
def extract_user_prompt_information(dataset, user_description):
    # Recupéartion des genres uniques dataset
    genres_available = dataset['Genre'].unique()
    doc = nlp(user_description)

    movie_info = {
        'Title': None,
        'Genres': [],
        'Release Year': None
    }

    for ent in doc.ents:
        print(f"Entité: {ent.text}, Catégorie: {ent.label_}")

        if ent.label_ == 'FILM' or ent.label_ == 'PERSON':
            movie_info['Title'] = ent.text

    # Utiliser l'expression régulière pour extraire le genre
    for genre in genres_available:
        if re.search(fr'\b{re.escape(genre.lower())}\b', user_description, flags=re.IGNORECASE):
            movie_info['Genres'].append(genre)

    # Utiliser l'expression régulière pour extraire l'année
    annee_match = re.search(r'\b(19[0-9][0-9]|200[0-9]|201[0-7])\b', user_description)
    if annee_match:
        movie_info['Release Year'] = annee_match.group()

    return movie_info

# Génération de la réponse pour la recommandation

In [ ]:
# Formulation de la réponse avec GPT-2
# Récupération du prompt (= les k films), du model gpt2 et du tokenizer
# Charger le modèle GPT-2 pré-entraîné et le tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
tokenizerGPT2 = GPT2Tokenizer.from_pretrained(model_name)
modelGPT2 = GPT2LMHeadModel.from_pretrained(model_name)

def generate_response_with_gpt2(prompt, model, tokenizer, max_length=1000):
    # Tokeniser le prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=max_length, truncation=True)

    # Créer l'attention mask
    attention_mask = input_ids.clone()
    attention_mask[attention_mask != tokenizer.pad_token_id] = 1  # Mettre à 1 tous les tokens qui ne sont pas des tokens de remplissage (padding)

    # Générer la réponse
    output = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7, pad_token_id=tokenizer.eos_token_id, attention_mask=attention_mask)
    # Décoder la réponse
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_response

# Exemple d'utilisation
user_prompt = "I'm in the mood for a movie. Any recommendations?"
generated_response = generate_response_with_gpt2(user_prompt, modelGPT2, tokenizerGPT2)

print(generated_response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


I'm in the mood for a movie. Any recommendations?


In [ ]:
import spacy
import re
nlp = spacy.load('en_core_web_sm')

def generate_final_recommendation(bert_model, dataset, filtre, genre, year, user_query):
    # Filtrage notre dataset par rapport au user_prompt
    # genre --> year --> titre ou description
    recommended_movies = []
    filtered_dataset = dataset

    # Si l'utilisateur a skip les 2 questions de genre et d'année
    # Appel de la fonction extract_user_prompt_information pour essayer de filtrer le dataset
    if genre == "" and year == "":
        user_desc_info = extract_user_prompt_information(dataset, user_query)
        # Pour le debugage
        print(user_desc_info)

        # Partie des filtres du dataset
        # Filtrage du dataset par rapport aux genres identifiés dans extract_user_prompt_information
        if user_desc_info['Genres'] is not None:
            for genres_identified in user_desc_info['Genres']:
                filtered_dataset = filter_by_genre_film(dataset, genres_identified)

        # Ce filtre ok Valide

    # Si le filtre d'année n'est pas vide
    # Appel de la fonction get_information_operator_and_year_from_prompt pour filtrer par rapport à l'année
    if year != "":
        filtered_dataset = get_information_operator_and_year_from_prompt(dataset, year)

    # Si le filtre de genre n'est pas vide
    # Appel de la fonction filter_by_genre_film pour filtrer par rapport au genre
    if genre != "":
        filtered_dataset = filter_by_genre_film(filtered_dataset, genre)

    # Fin de la partie des filtres
    # Recupérer les k films les plus proches de la descriptions utilisateur

    # Si l'utilisateur choisi de trouver des films par un titre de référence
    # On cherche dans le dataset le film correspondant
    # Si le film est trouver on récupère son ID sinon on renvoi un message d'excuse
    if filtre == "title":
        movie_id = filter_by_reference_title(dataset, user_query)
        if movie_id == "":
            return "Sorry, we can't find a movie"

        # Récupération du plot du film
        # Appel à la fonction find_k_films_bert_similarity_with_user_desc avec ce plot (movie_plot) pour l'embedding
        movie_plot = dataset.loc[movie_id, 'Plot']
        recommended_movies = find_k_films_bert_similarity_with_user_desc(bert_model, filtered_dataset, movie_plot, k=5)

    # Si l'utilisateur choisi de trouver des films par une description
    # Appel à la fonction find_k_films_bert_similarity_with_user_desc avec la descr user (user_query) pour l'embedding
    elif filtre == "description":
        recommended_movies = find_k_films_bert_similarity_with_user_desc(bert_model, filtered_dataset, user_query, k=5)

    # Une fois les k films trouvés
    # Formater les recommandations pour GPT-2
    formatted_recommendations = ", ".join([film['Title'] for film in recommended_movies])

    # Soumettre la description avec les recommandations à GPT-2 et extraire la réponse générée
    generated_response = generate_response_with_gpt2(f"Watch movies like {formatted_recommendations}", modelGPT2,
                                                     tokenizerGPT2)

    # Pour le débugage
    print(str(recommended_movies))

    # Retourne la recommandation génerée
    return generated_response

# Exemples


In [ ]:
test1 = generate_final_recommendation(modelBert, dataset, "description", "", "before 2010", "After burying Dobby, Harry Potter asks the goblin Griphook to help him, Ron, and Hermione break into Bellatrix Lestrange's vault at Gringotts bank, suspecting a Horcrux may be there. Griphook agrees, in exchange for the Sword of Gryffindor. ")
print(test1)

Identified dates:
[2010]


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[{'Title': 'Harry Potter and the Chamber of Secrets', 'Genre': 'family, fantasy', 'Director': 'Chris Columbus', 'Release Year': 2002, 'Similarity Score': 0.5967760621118702}, {'Title': 'Harry Potter and the Prisoner of Azkaban', 'Genre': 'family, fantasy', 'Director': 'Alfonso Cuarón', 'Release Year': 2004, 'Similarity Score': 0.5197679693757529}, {'Title': 'Love at Large', 'Genre': 'romance', 'Director': 'Alan Rudolph', 'Release Year': 1990, 'Similarity Score': 0.4835831470061166}, {'Title': '99 and 44/100% Dead', 'Genre': 'unknown', 'Director': 'John Frankenheimer', 'Release Year': 1974, 'Similarity Score': 0.4798205419312863}, {'Title': 'Harry Potter and the Order of the Phoenix', 'Genre': 'family, fantasy', 'Director': 'David Yates', 'Release Year': 2007, 'Similarity Score': 0.47593236066792777}]
Watch movies like Harry Potter and the Chamber of Secrets, Harry Potter and the Prisoner of Azkaban, Love at Large, 99 and 44/100% Dead, Harry Potter and the Order of the Phoenix, Deathly 

In [ ]:
test2 = generate_final_recommendation(modelBert, dataset, "A title", "fantasy", "After 2001", "Harry Potter")
print(test2)

Identified dates:
[2001]


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[]
Watch movies like  The Hunger Games: Mockingjay - Part 1 and  The Last Airbender: A Star Wars Story, and you'll be able to watch them all in one go.
But what if you don't want to go to the movies? Well, there are a few things you can do to make sure you're not going to miss out on some of the best movies of all time. Here's what you need to know:
1. You'll need an Internet connection
If you have an internet connection, you won't have to worry about downloading any of these movies. They're available on Netflix, Hulu, Amazon Instant Video, and other streaming services. If you've never watched a movie online before, it's a great way to get a taste of what's going on in the world of movies and TV. It's also great for those of us who like to binge watch movies on the go, or who just want a quick refresher on what movies are and aren't available in our local library. And if that's not enough for you, here are some tips to help you get the most out of your Netflix or Hulu subscription.
